# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [25]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd

# criar o navegador
nav = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [26]:
produto = 'iphone 12 64 gb'
lista_produto = produto.split(" ")
print(lista_produto)
print(produto)

['iphone', '12', '64', 'gb']
iphone 12 64 gb


In [27]:
nav.get("https://www.google.com/")
#nav.maximize_window()

produto = 'iphone 12 64 gb'
produto = produto.lower()
lista_termos_nome_produto = produto.split(" ")

termos_banidos = "mini watch"
termos_banidos = termos_banidos.lower()
lista_termos_banidos = termos_banidos.split(" ")

preco_minimo = 3000
preco_maximo = 4000
preco_minimo = float(preco_minimo)
preco_maximo = float(preco_maximo)


#Pesquisar pelo produto:
nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto)
nav.find_element('xpath', '//*[@id="APjFqb"]').send_keys(Keys.ENTER)


#Entrar na aba shopping:
elementos = nav.find_elements("class name", "hdtb-mitem")

for elemento in elementos:
    if "Shopping" in elemento.text: 
        elemento.click()
        break
        
#Pegar informações do produto
lista_resultados = nav.find_elements("class name", "i0X6df")

for resultado in lista_resultados:
    nome = resultado.find_element("class name", "tAxDx").text
    nome = nome.lower()
    
    # analisar se ele não tem nenhum termo banido
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    
    
    # analisar se ele tem TODOS os termos do nome do produto
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
                
    # Selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = True
    #if tem_termos_banidos == False and tem_todos_termos_produtos == True:
    if not tem_termos_banidos and tem_todos_termos_produtos:
        preco = resultado.find_element("class name", "a8Pemb").text
        preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
        preco = float(preco)
        
        # Verificar se o preço está dentro do preço minimo e máximo
        if preco_minimo <= preco <= preco_maximo:
            #Lógica para pegar o link, através do parâmetro "href", não foi possível Dessa a variavel elemento_referencia recebe a ("class name", "bONr3b"), que é childre do linkE pegar o "href" atraves do parent "("xpath", "..")"
            elemento_referencia = resultado.find_element("class name", "bONr3b")
            elemento_pai = elemento_referencia.find_element("xpath", "..")
            link = elemento_pai.get_attribute('href')
            print(preco, nome, link)
            
 



 
# #Pesquisar os preços
# precos = nav.find_elements("class name", "a8Pemb")
# for preco in precos:
#     print(preco.text)
        
# # pesquisar o nome do produto no google


# nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
# nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
# # clicar na aba shopping
# elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
# for item in elementos:
#     if "Shopping" in item.text:
#         item.click()
#         break    
    

3286.05 vitrine: iphone 12 apple 64gb azul tela 6,1 câmera traseira dupla 12mp https://www.google.com/url?url=https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-azul-tela-61-camera-traseira-dupla-12mp-mp911874363/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwifu9u8rr3-AhUIr5UCHTYgAwcQguUECO4M&usg=AOvVaw1RPVlB0_Sxvz71jhDvCmZP
3059.1 iphone 12 64gb - roxo - estou zerado https://www.google.com/url?url=https://www.trocafy.com.br/iphone-12-64gb-roxo-sou-como-novo-2555/p%3Fidsku%3D979%26srsltid%3DAfAwrE56QonpaC-DblVQYOopJzcuCoD-OCmhlnsgru4cVqXlyFA0Yo97dIc&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwifu9u8rr3-AhUIr5UCHTYgAwcQguUECJIN&usg=AOvVaw29BUoZAOVTN18owWvfgOcL
3286.05 vitrine: iphone 12 apple 64gb preto tela 6,1 câmera traseira dupla 12mp ios https://www.google.com/url?url=https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-preto-tela-61-camera-traseira-dupla-12mp-ios-mp911874394/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwifu9u8rr3-AhUIr5UCHTYgAwcQguUECKUN&usg=AOvVaw0iqjyEc2aYmu0MYRGNuUCD
3275.09 vitrine a